<a href="https://colab.research.google.com/github/goutham-751/Sentiment-Analysis-IMDB-Reviews/blob/main/Sentiment_Analysis_IMDB_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
#Setting up environmental variables
kaggle_config=json.load(open('kaggle.json'))
print(kaggle_config)
os.environ['KAGGLE_USERNAME']=kaggle_config['username']
os.environ['KAGGLE_KEY']=kaggle_config['key']

{'username': 'barksout2706', 'key': '031217ae0731bc34548c7d1bc689d17b'}


In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 314MB/s]


In [6]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [7]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip',"r") as z:
  z.extractall()

In [8]:
!ls

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


In [9]:
data=pd.read_csv('/content/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
data.shape
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
data.replace({"positive":1,"negative":0},inplace=True)

<ipython-input-11-cc722fc45cd2>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"positive":1,"negative":0},inplace=True)


In [12]:
data.isnull().sum()#There are no null values

,0
review,0
sentiment,0


In [13]:
#Train and test data
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)
print(train_data.shape,test_data.shape)

(40000, 2) (10000, 2)


Data Pre-processing

In [14]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [15]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [16]:
Y_train=train_data['sentiment']
Y_test=test_data['sentiment']

Building the model

In [17]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=5000, input_length=200))
model.add(LSTM(units=128, dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=5,batch_size=64)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 264s 406ms/step - accuracy: 0.7498 - loss: 0.5004
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 256s 405ms/step - accuracy: 0.8841 - loss: 0.2791
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 401ms/step - accuracy: 0.9281 - loss: 0.1847
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 267s 409ms/step - accuracy: 0.9514 - loss: 0.1297
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 287s 450ms/step - accuracy: 0.9686 - loss: 0.0899


In [19]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"Loss ,{loss}")
print(f"Accuracy{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 136ms/step - accuracy: 0.8868 - loss: 0.3045
Loss ,0.30420631170272827
Accuracy0.8888000249862671


Predictive System

In [30]:
def predict_sentiment(review):
    sequence=tokenizer.texts_to_sequences([review])
    pad_seq=pad_sequences(sequence,maxlen=200)
    prediction=model.predict(pad_seq)
    if(prediction[0][0]>0.5):
      print("Positive Review")
    else:
      print("Negative Review")


In [31]:
#Trying out the model
predict_sentiment("The movie was not good.Shit movie..")
predict_sentiment("The movie was Excellent")
predict_sentiment("The movie was okayish")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Negative Review
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Positive Review
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Negative Review
